In [1]:
#import required packages
#使用ib_insync是一个能比较简单的提取数据的包（相比于ib_api自带的功能）
import pandas as pd
from ib_insync import *
util.startLoop()

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=False, rc=None)

In [2]:
#establish connection with IB TWS
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=100)

<IB connected to 127.0.0.1:7497 clientId=100>

In [56]:
# #因为我们现在使用的是demo mode，所以这里使用汇率作为取数据演示
# contract = Forex('USDJPY')
# bars = ib.reqHistoricalData(
#     contract, endDateTime='', durationStr='1 Y',
#     barSizeSetting='1 day', whatToShow='MIDPOINT', useRTH=True)

# # convert to pandas dataframe:
# df = util.df(bars)
# df.head()

In [57]:
contract = Stock('TSLA', 'SMART', 'USD')

ib.reqHeadTimeStamp(contract, whatToShow='TRADES', useRTH=True)

datetime.datetime(2010, 6, 29, 8, 0)

In [58]:
contract = Stock('QQQ', 'SMART', 'USD')
dt = '20200101'
barsList = []
while True:
    bars = ib.reqHistoricalData(
        contract,
        endDateTime=dt,
        durationStr='10 D',
        barSizeSetting='1 hour',
        whatToShow='ADJUSTED_LAST',
        useRTH=True,
#         formatDateb=1
    )
    if not bars:
        break
    barsList.append(bars)
    dt = bars[0].date
    print(dt)
# save to CSV file
allBars = [b for bars in reversed(barsList) for b in bars]
df = util.df(allBars)

Error 321, reqId 129: Error validating request.-'bS' : cause - End date not supported with adjusted last, contract: Stock(symbol='QQQ', exchange='SMART', currency='USD')


In [59]:
df.set_index('date', inplace=True)

AttributeError: 'NoneType' object has no attribute 'set_index'

In [ ]:
df[['open','high','low','close']].tail(390).plot(figsize = (15,7))
plt.title("SPY Live Minute Data")

In [ ]:
df.tail(390)

In [ ]:
Option?

In [ ]:
option = Option('SPY', '20200416', 250, 'C', 'SMART', multiplier=100)
calc = ib.calculateImpliedVolatility(
    option, optionPrice=3.35, underPrice=248.21)
print(calc)
# calc = ib.calculateOptionPrice(
# option, volatility=0.14, underPrice=525)
# print(calc)

In [ ]:
ib.calculateImpliedVolatility?

In [ ]:
#因为demo mode所以现在会出现报错，使用实盘账户的话，应该可以跑通
code = "TSLA"
contract = Stock(symbol=code, exchange="SMART", currency="USD")
bars = ib.reqHistoricalData(contract, endDateTime='20190314 18:30:00', durationStr='5 D',
        barSizeSetting='30 secs', whatToShow='MIDPOINT', useRTH=True)
 
# convert to pandas dataframe:
df = util.df(bars)
print(df[['date', 'open', 'high', 'low', 'close']])

In [ ]:
df

# Be careful of the following code that executes trade orders on IB

In [29]:
class SIGNALS():
	# 在MACD模型下的开平仓算法
	def macd_updown_signals(df, signal_spread_col, date_col, ticker_col):
	    ticker = df[ticker_col].values[-1]
	    last_date = df[date_col].values[-1]
	    df.dropna(inplace = True)
	    df.reset_index(inplace = True)
	    del df['index']
	    listLongShort = []
	    macd = df[signal_spread_col].values
	    
	    for i in range(1, len(df)):
	        last_date = df[date_col][i]
	        
	        if macd[i]>macd[i-1] and macd[i-1]<macd[i-2]:
	            
	            listLongShort.append("BUY")
	        #                          # The other way around
	        elif macd[i]<macd[i-1] and macd[i-1]>macd[i-2]:
	            listLongShort.append("SELL")
	        #                          # Do nothing if not satisfied
	        elif macd[i]<macd[i-1]:
	            listLongShort.append("HOLD SHORT")
	            
	        elif macd[i]>macd[i-1]:
	            listLongShort.append("HOLD LONG")        
	            
	    return ticker, last_date, listLongShort[-1]
	#     df['Advice'] = listLongShort
	    # The advice column means "Buy/Sell/Hold" at the end of this day or
	    #  at the beginning of the next day, since the market will be closed


	def macd_cross_signals(df, signal_spread_col, date_col, ticker_col):
	    ticker = df[ticker_col].values[-1]
	    last_date = df[date_col].values[-1]
	    listLongShort = []
	    listDate = []
	    macd = df[signal_spread_col].values
	    
	    for i in range(1, len(df)):
	        last_date = df[date_col][i]
	        #                          # If the MACD crosses the signal line upward
	        if macd[i] >0 and macd[i - 1] <0:
	            listLongShort.append("BUY")
	            listDate.append(last_date)
	        #                          # The other way around
	        elif macd[i] < 0 and macd[i - 1] >0:
	            listLongShort.append("SELL")
	            listDate.append(last_date)
	        #                          # Do nothing if not crossed
	        else: # 还要改，再增加点条件
	            listLongShort.append("HOLD")
	            listDate.append(last_date)
	#     print("Ticker: ", ticker)
	#     print("Last Date", listDate[-1])
	#     print("Last Signal", listLongShort[-1])
	    return ticker, last_date, listLongShort[-1]
	#     df['Advice'] = listLongShort
	    # The advice column means "Buy/Sell/Hold" at the end of this day or
	    #  at the beginning of the next day, since the market will be closed

	def macd_updown_ma_slope(df, signal_spread_col, date_col, ticker_col, price_col):
		ticker = df[ticker_col].values[-1]
		last_date = df[date_col].values[-1]
		listLongShort = []
		# listDate = []
		macd = df[signal_spread_col].values
		# Calculate MA5:
		ma5 = pd.Series(df[price_col]).rolling(window=5).mean()
		# Calculate MA10:
		ma10 = pd.Series(df[price_col]).rolling(window=10).mean()

		for i in range(1, len(df)):
			current_date = df[date_col][i]
			last_close = df[price_col][i]
			last_ma5 = ma5[i]
			last_ma10 = ma10[i]
			sec_last_ma10 = ma10[i-1]

			if macd[i]>macd[i-1] and macd[i-1]<macd[i-2] and last_close>last_ma5 and last_ma10>sec_last_ma10:
	            
				listLongShort.append("买入")
	        #                          # The other way around
			elif macd[i]<macd[i-1] and macd[i-1]>macd[i-2] and last_close<last_ma5:
				listLongShort.append("卖出")
	        #                          # Do nothing if not satisfied
			elif macd[i]<macd[i-1] and last_close<last_ma5:
				listLongShort.append("空头持有")
	            
			elif macd[i]>macd[i-1] and last_close>last_ma5:
				listLongShort.append("多头持有")   
			else:
				listLongShort.append("其他状态")

		return ticker, last_date, listLongShort[-1]


	def macd_updown_ma_slope_bias(df, signal_spread_col, date_col, ticker_col, price_col):
		ticker = df[ticker_col].values[-1]
		last_date = df[date_col].values[-1]
		listLongShort = []
		# listDate = []
		macd = df[signal_spread_col].values
		# Calculate MA5:
		ma5 = pd.Series(df[price_col]).rolling(window=5).mean()
		# Calculate MA10:
		ma10 = pd.Series(df[price_col]).rolling(window=10).mean()
		

		for i in range(1, len(df)):
			current_date = df[date_col][i]
			last_close = df[price_col][i]
			sec_last_close = df[price_col][i-1]
			last_ma5 = ma5[i]
			sec_last_ma5 = ma5[i-1]
			last_ma10 = ma10[i]
			sec_last_ma10 = ma10[i-1]
			last_bias = last_close/last_ma10
# 			print(listLongShort)

			# MACD向上 and 收盘价上穿5日均线 and 10日均线向上
			if macd[i]>macd[i-1] and last_close>last_ma5 and sec_last_close < sec_last_ma5 and last_ma10>sec_last_ma10:
				listLongShort.append("买入")
	        # 收盘价下穿5日均线 or BIAS >1.1
			if last_close<last_ma5 and sec_last_close > sec_last_ma5 or last_bias>1.1:
				listLongShort.append("卖出平仓")

			else:
				listLongShort.append("-")
# 			if listLongShort[i]=="买入" or listLongShort[i]=="多头持有":
# 				listLongShort.append("多头持有")				

# 			if listLongShort[i]=="卖出平仓" or listLongShort[i]=="空仓":
# 				listLongShort.append("空仓")	

		return ticker, last_date, listLongShort[-1]




	def cross_MA(df, date_col, ticker_col, price_col, MA_para):
		ticker = df[ticker_col].values[-1]
		last_date = df[date_col].values[-1]
		listLongShort = []		
		ma = pd.Series(df[price_col]).rolling(window=MA_para).mean()

		for i in range(1, len(df)):
			last_close = df[price_col][i]
			if last_close > ma[i] and last_close < ma[i-1]:
				listLongShort.append("上穿MA{}，买入")
			elif last_close < ma[i] and last_close > ma[i-1]:
				listLongShort.append("下穿MA{}，卖出")
			else:
				listLongShort.append("价格压在均线上，无操作")
		return ticker, last_date, listLongShort[-1]

In [30]:
class GENERATE_SIGNALS():

	def generate_signals_macd_updown(data_list, model_name, date_col, price_col, ticker_col, model_freq):
	    stock_list = []
	    date_list = []
	    signal_list = []
	    signal_df = pd.DataFrame()
	    signal_count = 1

	    for data in data_list:
	        try:
	            TA_df = PRICE_VOL_INDICATORS.MACD(data, "Adj Close", 12, 26, 9)
	            ticker, last_date, last_signal = SIGNALS.macd_updown_signals(TA_df, "MACD", date_col, ticker_col)
	            stock_list.append(ticker)
	            date_list.append(last_date)
	            signal_list.append(last_signal)
	            print ("Signals({}) Prepared for No.{} : {}".format(model_freq, signal_count, ticker))
	        except Exception as e:
	            print(e)
	        signal_count+=1

	    signal_df['Ticker'] = stock_list
	    signal_df['Last_Date'] = date_list
	    signal_df['Signal'] = signal_list
	    signal_df['model_name'] = model_name
	    signal_df['model_freq'] = model_freq
	    print("-------------------------")
	    print("All Done!")
	    return signal_df

	def generate_signals_macd_updown_ma_slope(data_list, model_name, date_col, price_col, ticker_col, model_freq):
	    stock_list = []
	    date_list = []
	    signal_list = []
	    signal_df = pd.DataFrame()
	    signal_count = 1

	    for data in data_list:
	        try:
	            TA_df = PRICE_VOL_INDICATORS.MACD(data, "Adj Close", 12, 26, 9)
	            ticker, last_date, last_signal = SIGNALS.macd_updown_ma_slope(TA_df, "MACD", date_col, ticker_col, price_col)
	            stock_list.append(ticker)
	            date_list.append(last_date)
	            signal_list.append(last_signal)
	            print ("Signals({}) Prepared for No.{} : {}".format(model_freq, signal_count, ticker))
	        except Exception as e:
	            print(e)
	        signal_count+=1

	    signal_df['Ticker'] = stock_list
	    signal_df['Last_Date'] = date_list
	    signal_df['Signal'] = signal_list
	    signal_df['model_name'] = model_name
	    signal_df['model_freq'] = model_freq
	    print("-------------------------")
	    print("All Done!")
	    return signal_df


	def generate_signals_macd_updown_ma_slope_bias(data_list, model_name, date_col, price_col, ticker_col, model_freq):
	    stock_list = []
	    date_list = []
	    signal_list = []
	    signal_df = pd.DataFrame()
	    signal_count = 1

	    for data in data_list:
	        try:
	            TA_df = PRICE_VOL_INDICATORS.MACD(data, "Adj Close", 12, 26, 9)
	            ticker, last_date, last_signal = SIGNALS.macd_updown_ma_slope_bias(TA_df, "MACD", date_col, ticker_col, price_col)
	            stock_list.append(ticker)
	            date_list.append(last_date)
	            signal_list.append(last_signal)
	            print ("Signals({}) Prepared for No.{} : {}".format(model_freq, signal_count, ticker))
	        except Exception as e:
	            print(e)
	        signal_count+=1

	    signal_df['Ticker'] = stock_list
	    signal_df['Last_Date'] = date_list
	    signal_df['Signal'] = signal_list
	    signal_df['model_name'] = model_name
	    signal_df['model_freq'] = model_freq
	    print("-------------------------")
	    print("All Done!")
	    return signal_df


In [13]:
# 导入自己的模块
from data_download_US_HK import *
from strategies import *
from positions import *

In [31]:
ticker_list = ["ES=F","NQ=F","YM=F"]

YH_IB_Ticker_Map_US_Index_Futures = {"ES=F":"ES",
                       "YM=F":"YM",
                       "NQ=F":"NQ"}

us_db_path = "C:/Users/Administrator/CE_github_2/data_pipeline/Data/"

data_list_selected = LOAD_DATA.read_data_from_folders(ticker_list, us_db_path)

# Generate signals
daily_macd_updown_signals = GENERATE_SIGNALS.generate_signals_macd_updown_ma_slope_bias( 
                                                                       data_list_selected,
#                                                                            CN_fund_data,
                                                                      "Macd_Updown_Ma_Slope_Bias", 
                                                                      "Date", 
                                                                      "Adj Close", 
                                                                      "Ticker", 
                                                                      "D")

In [40]:
# futures
for ticker in ticker_list:
    print("检测{}信号...".format(ticker))
    if str(daily_macd_updown_signals[daily_macd_updown_signals['Ticker'] == ticker]['Signal'].values[0])=="买入":
        print("Signal for: {}".format(ticker),":买入")
        IB_ticker = YH_IB_Ticker_Map_US_Index_Futures[ticker]
        
        contract = Future(IB_ticker, '20210618', 'GLOBEX')
        ib.qualifyContracts(contract)
        # step 2 定义合约单信息
        order = MarketOrder('BUY', 1)

        # step 3 定义交易功能
        trade = ib.placeOrder(contract, order)

        # print(trade)

        ib.sleep(1)
        print("Bought")
        
    if str(daily_macd_updown_signals[daily_macd_updown_signals['Ticker'] == ticker]['Signal'].values[0])=="卖出平仓":
        print("Signal for: {}".format(ticker),":卖出平仓")
        IB_ticker = YH_IB_Ticker_Map_US_Index_Futures[ticker]
        
        contract = Future(IB_ticker, '20210618', 'GLOBEX')
        ib.qualifyContracts(contract)
        # step 2 定义合约单信息
        order = MarketOrder('SELL', 1)

        # step 3 定义交易功能
        trade = ib.placeOrder(contract, order)

        # print(trade)

        ib.sleep(1)
        print("SELL")       
        
    else:
        print("无操作")
    print("--------------")

检测ES=F信号...
无操作
--------------
检测NQ=F信号...
无操作
--------------
检测YM=F信号...
无操作
--------------


In [53]:
ticker_list = list(set(['9988.HK','6618.HK','3888.HK','0418.HK',
		'9999.HK','1909.HK','2318.HK','0941.HK','0005.HK','9999.HK',
		'1070.HK','6969.HK','0241.HK','1137.HK','6055.HK','6862.HK',
		'0700.HK','2051.HK','6993.HK','6618.HK','9992.HK','3690.HK',
		'1810.HK','1024.HK','1119.HK','8029.HK','0493.HK','2382.HK',
		'0175.HK','6969.HK','1610.HK','2400.HK','0788.HK','0981.HK',
		'0728.HK','0762.HK','1833.HK','0285.HK','0268.HK','0763.HK',
		'6823.HK','6060.HK','0151.HK','1347.HK','3799.HK','2018.HK',
		'0522.HK','0008.HK','2400.HK','0552.HK','1458.HK','6088.HK',
		'1478.HK','0303.HK','1070.HK','1310.HK','9990.HK','0799.HK',
		'3738.HK','1675.HK','0142.HK','0777.HK','1883.HK','2038.HK',
		'0419.HK','6869.HK','1385.HK','0302.HK','0797.HK','3798.HK',
		'0751.HK','1475.HK','0215.HK','2342.HK','2100.HK','0315.HK',
		'6820.HK','1909.HK','1415.HK','8032.HK','1119.HK','0732.HK',
		'1588.HK','2309.HK','0698.HK','2369.HK','0596.HK','1566.HK',
		'0186.HK','2255.HK','0439.HK','6188.HK','1979.HK']))

ticker_list = ticker_list[:10]

In [51]:
YH_IB_Ticker_Map_HK_Stocks = {}
for ticker in ticker_list[:10]:
    IB_ticker = ticker[:4]
    YH_IB_Ticker_Map_HK_Stocks[ticker] = IB_ticker
#     print(IB_ticker)

In [52]:
YH_IB_Ticker_Map_HK_Stocks

{'6823.HK': '6823',
 '6869.HK': '6869',
 '1478.HK': '1478',
 '0552.HK': '0552',
 '9999.HK': '9999',
 '1458.HK': '1458',
 '1385.HK': '1385',
 '6088.HK': '6088',
 '0005.HK': '0005',
 '0008.HK': '0008'}

In [56]:
101.77/181.80-1

-0.44020902090209024

In [54]:
422+1693+401

2516

In [ ]:
us_db_path = "C:/Users/Administrator/CE_github_2/data_pipeline/Data/"

data_list_selected = LOAD_DATA.read_data_from_folders(ticker_list, us_db_path)

# Generate signals
daily_macd_updown_signals = GENERATE_SIGNALS.generate_signals_macd_updown_ma_slope_bias( 
                                                                       data_list_selected,
#                                                                            CN_fund_data,
                                                                      "Macd_Updown_Ma_Slope_Bias", 
                                                                      "Date", 
                                                                      "Adj Close", 
                                                                      "Ticker", 
                                                                      "D")

In [ ]:
# HK stocks:

for ticker in ticker_list:
    print("检测{}信号...".format(ticker))
    if str(daily_macd_updown_signals[daily_macd_updown_signals['Ticker'] == ticker]['Signal'].values[0])=="买入":
        print("Signal for: {}".format(ticker),":买入")
        IB_ticker = YH_IB_Ticker_Map_US_Index_Futures[ticker]
        
        contract = Future(IB_ticker, '20210618', 'GLOBEX')
        ib.qualifyContracts(contract)
        # step 2 定义合约单信息
        order = MarketOrder('BUY', 1)

        # step 3 定义交易功能
        trade = ib.placeOrder(contract, order)

        # print(trade)

        ib.sleep(1)
        print("Bought")
        
    if str(daily_macd_updown_signals[daily_macd_updown_signals['Ticker'] == ticker]['Signal'].values[0])=="卖出平仓":
        print("Signal for: {}".format(ticker),":卖出平仓")
        IB_ticker = YH_IB_Ticker_Map_US_Index_Futures[ticker]
        
        contract = Future(IB_ticker, '20210618', 'GLOBEX')
        ib.qualifyContracts(contract)
        # step 2 定义合约单信息
        order = MarketOrder('SELL', 1)

        # step 3 定义交易功能
        trade = ib.placeOrder(contract, order)

        # print(trade)

        ib.sleep(1)
        print("SELL")       
        
    else:
        print("无操作")
    print("--------------")

In [ ]:
# 这是一个使用ib_sync和ib_api 来实现下单的一个例子

# step 1 定义、验证合约有效性
# contract = Stock('3690', 'SEHK', 'HKD')

# assert trade in ib.trades()
# assert order in ib.orders()

# 期货example:
# contract = Future('ES', '20210618', 'GLOBEX')

# 港股example:
contract = Stock('6618', exchange='SEHK')
ib.qualifyContracts(contract)

# step 2 定义合约单信息
order = MarketOrder('BUY', 50)

# step 3 定义交易功能
trade = ib.placeOrder(contract, order)

# print(trade)

ib.sleep(1)
# print(trade.log)

# limitOrder = LimitOrder('BUY', 1, 0.05)
# limitTrade = ib.placeOrder(contract, limitOrder)

# print(limitTrade)
# ib.placeOrder(contract, limitOrder)
# ib.cancelOrder(limitOrder)

In [ ]:
contract = Stock('GOOGL', exchange="SMART", currency="USD")
ib.qualifyContracts(contract)

# step 2 定义合约单信息
order = MarketOrder('BUY', 1)

# step 3 定义交易功能
trade = ib.placeOrder(contract, order)

# print(trade)

ib.sleep(1)

In [ ]:
#尝试调取option数据
spx = Index('SPX')
ib.qualifyContracts(spx)
chains= ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

In [ ]:
options_chain = util.df(chains)

In [ ]:
options_chain

In [ ]:
contract = Stock('IBM', 'SMART', 'USD') 
ticker = ib.reqMktData(contract, '258') 
ib.sleep(2) 
print(ticker.fundamentalRatios)

In [ ]:
#由于是demo mode 所以报错了，TWS setup 文件里最后一个链接中有具体的demo
[ticker] = ib.reqTickers(spx)
ticker
spxValue = ticker.marketPrice()
spxValue

chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P', 'C']

contracts = [Option('SPX', expiration, strike, right, 'SMART', tradingClass='SPX')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*contracts)
len(contracts)

In [ ]:
contracts